# 02-02 - Azure Cosmos DB in deiner Resource Group erstellen

## Überblick
In diesem Notebook erfährst du, wie du **Azure Cosmos DB** für NoSQL erstellst und konfigurierst – ideal für Test- oder Hackathon-Umgebungen. Wir gehen folgende Punkte durch:
1. **Erstellung einer neuen Cosmos DB-Instanz (NoSQL)** über das **Azure Portal**.
2. **Optionale CLI-Schritte** für Netzwerkzugang und lokale Authentifizierung (Primär-/Sekundärschlüssel).
3. **Datenbank und Container** im Data Explorer anlegen.
4. **.env-Konfiguration** mit Cosmos DB URL, Key, etc.
5. **Beispiel:** Daten (Filmliste) aus CSV laden, Embeddings generieren und anschließend in Cosmos DB hochladen.


## 1) Neue Azure Cosmos DB-Instanz erstellen

In dieser Anleitung zeigen wir dir Schritt für Schritt, wie du eine neue Azure Cosmos DB-Instanz in deiner Resource Group erstellst – unter Nutzung der **Azure Cosmos DB für NoSQL**.

### Eine neue Azure Cosmos DB-Instanz erstellen
Öffne das **Azure Portal** und suche im **Marketplace** nach **Cosmos DB**. Wähle den entsprechenden Eintrag aus.

![CosmosDB suchen](images/find-cosmosdb.png)

### NoSQL als API-Typ wählen
Im Schritt „Empfohlene APIs“ wähle **Azure Cosmos DB für NoSQL**.

#### ❓ Was ist NoSQL?
NoSQL-Datenbanken sind flexible, schemafreie Datenbanken, die besonders gut für skalierbare Anwendungen geeignet sind. Im Gegensatz zu relationalen SQL-Datenbanken benötigen NoSQL-DBs keine festen Tabellenstrukturen und sind so optimal für dynamische Anwendungen.

### Einrichten der Instanz
- **Account-Name**: Wähle einen **eindeutigen Namen** für deine Cosmos DB Instanz.
- **Region**: Wähle z. B. **Switzerland North** als Standort, falls gewünscht.

![CosmosDB erstellen](images/create-cosmosdb.png)

### Availability Zones deaktivieren
Setze **Availability Zones** auf **Deaktiviert**, da wir hier nur **Nicht-Produktionsdaten** speichern. Für produktive Umgebungen sollte man **Aktiviert** wählen.

### Kapazitätsmodus: Serverless
Wähle **Serverless** als Kapazitätsmodus.

#### ❓ Warum nicht „Provisioned Throughput“?
- Für Test- oder Hackathon-Umgebungen sind die Nutzungszeiten unregelmäßig.
- „Serverless“ kostet nur bei tatsächlicher Nutzung, anstatt kontinuierliche Kapazität zu reservieren.

### Netzwerkkonfiguration
Im **Netzwerk-Tab**: "Datenverkehr von allen Netzwerken zulassen", wenn jeder Teilnehmer Zugriff haben soll.

### Service erstellen
Lasse die restlichen Einstellungen unverändert und erstelle die Cosmos DB Instanz. Nach ein paar Minuten ist sie einsatzbereit!


## 2) Anmeldung bei Azure in der Konsole (optional)

Um sich bei Azure über die Konsole anzumelden, verwende:
```sh
az login
```
Falls du mehrere Mandanten (Tenants) hast und dich gezielt bei einem bestimmten anmelden möchtest:
```sh
az login --tenant <TENANT_ID>
```

Beispiel:
```sh
az login --tenant 119cbc86-5275-4878-8321-4d8da34a0dc2
```
Dadurch wird sichergestellt, dass du dich im gewünschten Azure Active Directory (AAD) befindest.

## 3) Öffentlichen Netzwerkzugang aktivieren & Zugriff mit Primär-/Sekundärschlüssel ermöglichen

Wir setzen voraus, dass deine Cosmos DB schon existiert, jedoch standardmäßig kein öffentlicher Zugriff möglich ist.

```sh
az cosmosdb update \
    --name "<COSMOSDB_NAME>" \
    --resource-group "<RESOURCE_GROUP>" \
    --public-network-access ENABLED
```

Zusätzlich prüfst du, ob `disableLocalAuth` auf `false` steht:
```sh
az cosmosdb show \
    --name "<COSMOSDB_NAME>" \
    --resource-group "<RESOURCE_GROUP>" \
    --query "disableLocalAuth"
```
Sobald es `false` ist, kannst du den Primär-/Sekundärschlüssel (Keys) verwenden, um Zugriff zu erhalten.

## 4) Neue Datenbank und Container erstellen

### Data Explorer (Portal)
1. Navigiere im Azure-Portal zur CosmosDB-Instanz.
2. Klicke auf **Data Explorer**.
3. **Neuer Container**
   - **Datenbank-ID**: Name der neuen DB
   - **Container-ID**: Name des Containers
   - **Partition Key**: z. B. `/id`
   - **Unique Key**: optional, z. B. `/id`

![Neue Datenbank und Container erstellen](images/new-container.png)

### Verbindung per .env
1. Öffne die Cosmos DB-Übersicht → **Keys**-Tab
   ![Primärschlüssel](images/key-and-url.png)
2. Notiere den **Primärschlüssel** und die **URI**.
3. Setze sie in deiner `.env`:
   ```bash
   COSMOSDB_URL="<URI_AUS_AZURE_PORTAL>"
   COSMOSDB_KEY="<PRIMÄRSCHLÜSSEL_AUS_AZURE_PORTAL>"
   COSMOSDB_NAME="<NAME_DER_ERSTELLTEN_DB>"
   COSMOSDB_CONTAINER_NAME="<NAME_DES_CONTAINERS>"
   ```

## 5) Beispiel: Cosmos DB befüllen

Wir verbinden uns programmgesteuert mit unserer Cosmos DB und erstellen – falls nicht vorhanden – die Datenbank und den Container. Anschließend laden wir Daten (z. B. Filmdaten) aus einer CSV-Datei, generieren Embeddings via Azure OpenAI und speichern alles in unserem Cosmos DB Container.

### 5.1) Verbindungsaufbau mit Cosmos DB

In [ ]:
# Import der benötigten Bibliotheken
from azure.cosmos import CosmosClient, PartitionKey
import os
from dotenv import load_dotenv

# Laden der Umgebungsvariablen
load_dotenv()

COSMOS_DB_URL = os.getenv("COSMOSDB_URL")
COSMOS_DB_KEY = os.getenv("COSMOSDB_KEY")
DATABASE_NAME = os.getenv("COSMOSDB_NAME")
CONTAINER_NAME = os.getenv("COSMOSDB_CONTAINER_NAME")

# CosmosClient erstellen
client = CosmosClient(COSMOS_DB_URL, COSMOS_DB_KEY)
database = client.create_database_if_not_exists(DATABASE_NAME)
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id")
)

print("CosmosDB connection successful and container initialized!")

### 5.2) Embeddings via Azure OpenAI

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

def generate_embedding(content: str):
    try:
        return azure_openai_embeddings.embed_query(content)
    except Exception as e:
        print(f"Fehler beim Erzeugen des Embeddings: {e}")
        return []

### 5.3) CSV-Daten laden (z. B. Filmliste) und parsieren

Wir laden mit der `CSVLoader`-Klasse (aus `langchain.document_loaders`) eine CSV-Datei (`movies.csv`). Anschließend reduzieren wir die Liste auf 50 Einträge, erzeugen jeweils ein Embedding und parsen den Inhalt in ein Dictionary.

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path='./movies.csv',
    source_column='original_title',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'fieldnames': [
            'id', 'original_language', 'original_title', 'popularity',
            'release_date', 'genre',
            'overview', 'revenue', 'runtime', 'tagline'
        ]
    }
)
data = loader.load()
# Optional: Anzahl Datensätze beschränken
data = data[1:51]
print(f"Loaded {len(data)} movies.")

def parse_movie(movie, vector):
    try:
        content = movie.page_content
        fields = dict(line.split(": ", 1) for line in content.split("\n") if ": " in line)

        return {
            "id": int(float(fields.get("id", "0"))),
            "original_language": fields.get("original_language", "").strip(),
            "original_title": fields.get("original_title", "").strip(),
            "popularity": fields.get("popularity", "0"),
            "release_date": fields.get("release_date", "").strip(),
            "vote_average": fields.get("vote_average", "0"),
            "vote_count": fields.get("vote_count", "0"),
            "genre": fields.get("genre", "[]"),
            "overview": fields.get("overview", "").strip(),
            "revenue": fields.get("revenue", "0"),
            "runtime": fields.get("runtime", "0"),
            "tagline": fields.get("tagline", "").strip(),
            "vector": vector
        }
    except Exception as e:
        print(f"Fehler beim Parsen: {e}")
        return {}

parsed_movies = []
for m in data:
    vec = generate_embedding(m.page_content)
    parsed_movies.append(parse_movie(m, vec))

print("Erstes Beispiel:", parsed_movies[0])

### 5.4) Items in den Cosmos DB Container hochladen

Wir durchlaufen unsere vorbereiteten Filmdaten und erstellen für jedes Objekt einen Upsert in Cosmos DB. Dabei muss die ID als String vorliegen. Anschließend können wir im Data Explorer des Azure-Portals unsere Daten sehen.

In [ ]:
for item in parsed_movies:
    item["id"] = str(item["id"])
    try:
        container.upsert_item(item)
        print("Uploaded:", item["original_title"])
    except Exception as e:
        print(f"Fehler beim Hochladen von {item['original_title']}: {e}")

print(f"Successfully uploaded {len(parsed_movies)} movies into CosmosDB! 🚀")